In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz
event_name = "Apply halt: 2s" #event to align data to, can make into list in the future?

#PATH TO Cohort 1 visual mismatch THIS
rawdata_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])


photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")




In [ ]:
#---------------------------------------------------
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------

df_to_analyze = photometry_tracking_encoder_data["Photodiode_int"] #using downsampled values in common time grid 
#df_to_analyze = camera_photodiode_data["Photodiode"] #use async raw values if needed for troubleshooting, but the nearest indices needs to be found , see couple of lines below
photodiode_halts, photodiode_delay_min, photodiode_delay_avg, photodiode_delay_max = process.analyze_photodiode(df_to_analyze, experiment_events, event_name, plot = True)
# nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
# photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)

In [ ]:
# Force reload the modules
importlib.reload(harp_resources.process)
importlib.reload(harp_resources.utils)
# Reassign after reloading to ensure updated references
process = harp_resources.process
utils = harp_resources.utils

In [ ]:
#-----------------------------------------
# Get Block times and durations
#-----------------------------------------

block_events = experiment_events[
    experiment_events["Event"].str.contains("block started|Block timer elapsed", case=False, na=False)
].copy()  # Use .copy() to avoid SettingWithCopyWarning
block_events["Time Difference"] = block_events.index.to_series().diff().dt.total_seconds()
block_events = block_events.drop(columns=['Seconds'])

# Print results
print(block_events)


NEXT - get event triggered averages of all relevant data and reproduce figure below 
**THIS IS NOT WORKING, SUGGEST 1. downsample photodiode signal and well and save togerther with dataset.**

NEXT - think about / run some correlation analysis 

In [ ]:
### Don't calculate velocity, but use Motor_Velocity column 
### add baselining to all data before new plot 
### quantify peak fluorescnce 
### quantify motor velocity vs animal velocity correlation - is there a lag?
### LOOK INTO THE superbig filtering in the velocity and ACC calculations (looks like it's 1 Hz???)

In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

# Define time window variables
time_window_start = -2  # Modify this to change the start time
time_window_end = 6  # Modify this to change the end time

# Create an empty DataFrame to store aligned data
aligned_data = []

# Loop through each halt event time
for halt_time in photodiode_halts:
    # Extract data within the selected time window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time + pd.Timedelta(seconds=time_window_start)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=time_window_end))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows
aligned_df = pd.concat(aligned_data, ignore_index=True)

# Unwrap the Encoder signal to remove discontinuities
aligned_df["Unwrapped_Encoder"] = np.unwrap(np.deg2rad(aligned_df["Encoder"]))  
aligned_df["Unwrapped_Encoder"] = np.rad2deg(aligned_df["Unwrapped_Encoder"])  

# Compute motor velocity (derivative of unwrapped encoder)
aligned_df["Motor_Velocity"] = aligned_df["Unwrapped_Encoder"].diff() / aligned_df["Time (s)"].diff()

# Apply a 100 Hz low-pass Butterworth filter to smooth motor velocity
fs = 1000  
cutoff = 100  
b, a = signal.butter(2, cutoff / (fs / 2), btype='low', analog=False)
aligned_df["Motor_Velocity_Smoothed"] = signal.filtfilt(b, a, aligned_df["Motor_Velocity"].fillna(0))

# Compute velocity from Position_0X and Position_0Y, and scale Velocity_0X
aligned_df["Velocity_0X"] = aligned_df["Position_0X"].diff() / aligned_df["Time (s)"].diff() / 1000  
aligned_df["Velocity_0Y"] = aligned_df["Position_0Y"].diff() / aligned_df["Time (s)"].diff()

# Compute mean and standard error of the mean (SEM)
mean_df = aligned_df.groupby("Time (s)").mean()
sem_df = aligned_df.groupby("Time (s)").sem()  

# Create figure for the two plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True)

### PLOT 1: Individual Traces - Photodiode, z_470, z_560 ###
ax1 = axes[0]

for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1.plot(subset["Time (s)"], subset["Photodiode_int"], color='grey', alpha=0.5)

ax1.set_xlabel('Time (s) relative to halt')
ax1.set_ylabel('Photodiode_int')
ax1.set_title('Photodiode, z_470, and z_560')

ax1_2 = ax1.twinx()
for halt_time in photodiode_halts:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1_2.plot(subset["Time (s)"], subset["z_470"], color='green', linestyle='-', alpha=0.5)
    ax1_2.plot(subset["Time (s)"], subset["z_560"], color='red', linestyle='-', alpha=0.5)

ax1_2.set_ylabel('Fluorescence (z_470: Green, z_560: Red)')

### PLOT 2: Mean & SEM of All Signals ###
ax2 = axes[1]

ax2.plot(mean_df.index, mean_df["Photodiode_int"], color='grey', alpha=0.8)
ax2.fill_between(mean_df.index, mean_df["Photodiode_int"] - sem_df["Photodiode_int"], 
                 mean_df["Photodiode_int"] + sem_df["Photodiode_int"], color='grey', alpha=0.2)

ax2.set_xlabel('Time (s) relative to halt')
ax2.set_ylabel('Photodiode_int')
ax2.set_title('Mean & SEM of All Signals')

ax2_2 = ax2.twinx()
ax2_2.plot(mean_df.index, mean_df["z_470"], color='green', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_470"] - sem_df["z_470"], 
                    mean_df["z_470"] + sem_df["z_470"], color='green', alpha=0.2)

ax2_2.plot(mean_df.index, mean_df["z_560"], color='red', linestyle='-', alpha=0.8)
ax2_2.fill_between(mean_df.index, mean_df["z_560"] - sem_df["z_560"], 
                    mean_df["z_560"] + sem_df["z_560"], color='red', alpha=0.2)

ax2_3 = ax2.twinx()
ax2_3.spines['right'].set_position(('outward', 50))  
ax2_3.plot(mean_df.index, mean_df["Motor_Velocity_Smoothed"], color='#00008B', linestyle='-', alpha=0.8)
ax2_3.fill_between(mean_df.index, mean_df["Motor_Velocity_Smoothed"] - sem_df["Motor_Velocity_Smoothed"], 
                    mean_df["Motor_Velocity_Smoothed"] + sem_df["Motor_Velocity_Smoothed"], color='#00008B', alpha=0.2)

ax2_3.set_ylabel('Motor Velocity (Dark Blue)')

ax2_4 = ax2.twinx()
ax2_4.spines['right'].set_position(('outward', 100))  
ax2_4.plot(mean_df.index, mean_df["Velocity_0X"], color='orange', linestyle='-', alpha=0.8)
ax2_4.fill_between(mean_df.index, mean_df["Velocity_0X"] - sem_df["Velocity_0X"], 
                    mean_df["Velocity_0X"] + sem_df["Velocity_0X"], color='orange', alpha=0.2)

ax2_4.set_ylabel('Velocity_0X (Orange) (scaled x1000)')

ax2_5 = ax2.twinx()
ax2_5.spines['right'].set_position(('outward', 150))  
ax2_5.plot(mean_df.index, mean_df["Velocity_0Y"], color='#ADD8E6', linestyle='-', alpha=0.8)
ax2_5.fill_between(mean_df.index, mean_df["Velocity_0Y"] - sem_df["Velocity_0Y"], 
                    mean_df["Velocity_0Y"] + sem_df["Velocity_0Y"], color='#ADD8E6', alpha=0.2)

ax2_5.set_ylabel('Velocity_0Y (Light Blue)')

# Adjust layout
fig.tight_layout()
plt.show()


In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
